In [1]:
!pip install mysql-connector-python sqlalchemy matplotlib
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from db_setup import setup_database

# Set up the MySQL database
engine = setup_database()

# Function to run a single query and return a DataFrame
def run_query(query, params=None):
    try:
        with engine.connect() as connection:
            return pd.read_sql(query, connection, params=params)
    except Exception as e:
        print(f"Error executing query: {query}\nError: {str(e)}")
        return None

# Function to run dynamic region query
def run_dynamic_region_query():
    valid_regions = ['Central', 'East', 'South', 'West']
    region = input("Enter a region (Central, East, South, West) : ")
    while region not in valid_regions:
        print("Invalid region! Please choose Central, East, South, or West")
        region = input("Enter a region (Central, East, South, West) : ")
    query = """
    SELECT DATE_FORMAT(order_date, '%Y-%m') AS month, ROUND(SUM(sales), 2) AS total_sales
    FROM super_store_data
    WHERE region = %s
    GROUP BY month
    ORDER BY month;
    """
    df = run_query(query, params=(region,))
    return df

# Function to run multiple queries from a file
def RUN_QUERIES(filename):
    with open(filename, 'r') as file:
        script = file.read()
        queries = [q.strip() for q in script.split(';') if q.strip()]  # Skipping empty queries
        result = []
        for query in queries:
            if '%s' not in query:  # Skipping dynamic query
                df = run_query(query)
                if df is not None:
                    result.append(df)
        return result

# Function to plot 3 (2 , 5,  8 ) query results
def plot_results(results):
    # Check if results list is valid
    if not results:
        print("No results to plot")
        return

    # Plot Query 2: Sales by category (Bar chart)
    if len(results) > 1 and not results[1].empty:
        try:
            categories = results[1]['category']
            sales = results[1]['SUM(sales)']
            plt.figure(figsize=(8, 5))
            plt.bar(categories, sales, color='blue')
            plt.title('Total Sales by Category')
            plt.xlabel('Category')
            plt.ylabel('Sales ($)')
            plt.tight_layout()
            plt.savefig('sales_by_category.png')
            plt.close()
            print("Saved sales_by_category.png")
        except Exception as e:
            print(f"Error plotting Query 2: {str(e)}")

    # Plot Query 5: Top 5 states by profit (Horizontal bar chart)
    if len(results) > 4 and not results[4].empty:
        try:
            top_5 = results[4].head(5)
            states = top_5['state']
            profits = top_5['SUM(profit)']
            plt.figure(figsize=(8, 5))
            plt.barh(states, profits, color='orange')
            plt.title('Top 5 States by Profit')
            plt.xlabel('Profit ($)')
            plt.ylabel('State')
            plt.tight_layout()
            plt.savefig('profit_by_state.png')
            plt.close()
            print("Saved profit_by_state.png")
        except Exception as e:
            print(f"Error plotting Query 5: {str(e)}")

    # Plot Query 8: Top 3 states by profit margin (Pie chart)
    if len(results) > 7 and not results[7].empty:
        try:
            states = results[7]['state']
            margins = results[7]['profit_margin']
            plt.figure(figsize=(6, 6))
            plt.pie(margins, labels=states, autopct='%1.1f%%', colors=['blue', 'orange', 'green'])
            plt.title('Top 3 States by Profit Margin')
            plt.tight_layout()
            plt.savefig('profit_margin_by_state.png')
            plt.close()
            print("Saved profit_margin_by_state.png")
        except Exception as e:
            print(f"Error plotting Query 8: {str(e)}")

# Run the queries
results = RUN_QUERIES("sql_queries.sql")

# Run the dynamic region query
dynamic_result = run_dynamic_region_query()
if dynamic_result is not None:
    results.append(dynamic_result)

# Print results
for i, df in enumerate(results):
    print(f"\nResult of query {i + 1}:")
    print(df)

# Plot the results
plot_results(results)

print("Analysis complete , Check sales_by_category.png , profit_by_state.png , and profit_margin_by_state.png in the project folder")


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
C:\Users\user\Desktop\datascience projects and practice\Sales Data Analysis & Visualization Project\db_setup.py:51: SyntaxWarning: invalid escape sequence '\S'
  csv_path = "data\Superstore-data.csv"


Enter MySQL username:  root
Enter MySQL password:  8596
Enter MySQL host (default: localhost):  localhost
Enter database name:  hashir


Data loaded into table successfully.
Error executing query: --Query 9: Dynamic sales by region and month, implemented in sales_analysis.ipynb

-- Query 10: Orders with high discounts
SELECT category, COUNT(DISTINCT order_id) AS high_discount_orders
FROM super_store_data
WHERE discount > 0.2
GROUP BY category
ORDER BY high_discount_orders DESC
Error: (mysql.connector.errors.ProgrammingError) 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '--Query 9: Dynamic sales by region and month, implemented in sales_analysis.ipyn' at line 1
[SQL: --Query 9: Dynamic sales by region and month, implemented in sales_analysis.ipynb

-- Query 10: Orders with high discounts
SELECT category, COUNT(DISTINCT order_id) AS high_discount_orders
FROM super_store_data
WHERE discount > 0.2
GROUP BY category
ORDER BY high_discount_orders DESC]
(Background on this error at: https://sqlalche.me/e/20/f405)


Enter a region (Central, East, South, West) :  West



Result of query 1:
    region   SUM(sales)
0    South  391721.9050
1     West  725457.8245
2  Central  501239.8908
3     East  678781.2400

Result of query 2:
          category   SUM(sales)
0        Furniture  741999.7953
1  Office Supplies  719047.0320
2       Technology  836154.0330

Result of query 3:
  sub_category   SUM(sales)
0    Bookcases  114879.9963
1       Chairs  328449.1030
2       Labels   12486.3120
3       Tables  206965.5320
4      Storage  223843.6080

Result of query 4:
   DATE_FORMAT(order_date, '%Y-%m')   SUM(sales)
0                           2016-11   79411.9658
1                           2016-06   40344.5340
2                           2015-10   31404.9235
3                           2014-06   34595.1276
4                           2017-04   36521.5361
5                           2016-12   96999.0430
6                           2015-11   75972.5635
7                           2014-11   78628.7167
8                           2014-05   23648.2870
9             